# Introduction

## Présentation

Redis est une base de données NoSQL qui se classe dans la catégorie "data structure server". C'est  un système permettant de traiter, de manière très performante, des données sous la forme clés/valeurs. Cette performance est obtenue car les données sont stockées directement sur la mémoire RAM. Cela permet d'accéder très facilement et très rapidement aux données 

La clé permet de récupérer et d'identifier de manière unique la donnée. Redis ne permet pas de stocker des modèles de données très complexes, il faut donc ruser pour créer une structure de données complète et complexe.

Redis est souvent utilisé dans le developpement web pour gérer les sessions utilisateurs, leur panier, leurs configurations. En outre, lorsqu'on stocke une petite quantité de donnée et qu'on a besoin d'y accèder en quelques milliseconds

## Les données 



Les valeurs peuvent appartenir aux types suivants :

- chaînes de caractères
- compteurs numériques (les données numériques seront stockées dans la base sous la forme de chaîne de caractères)
- listes 
- sets ou sets ordonnés
- dictionnaires

## Lancer un container Redis

Comme pour Mongo, il nous faut une instance redis qui tourne en local, pour se faire il suffit de lancer un container Redis

Vous devriez être familier avec Docker maintenant, à vous de jouer pour lancer ce container !

## Installation

Le package Redis pour python est déjà installé, voici la commande si jamais :

`pipenv install redis==5.0.8`

# Let's play

In [17]:
import redis

In [18]:
redis_client = redis.StrictRedis(host='localhost')

Pour tester la connexion on peut pinguer la base de données :

In [19]:
redis_client.ping()

True

Vous pouvez regarder toutes les configurations utilisées par Redis, beaucoup ne sont pas utiles à notre niveau, mais vous pouvez voir par exemple, `database`, `maxclients`, `port` , `timeout` qui peuvent être utile de modifier pour différentes applications

In [20]:
redis_client.config_get()

{'appendfsync': 'everysec',
 'sanitize-dump-payload': 'no',
 'server-cpulist': '',
 'stream-node-max-entries': '100',
 'client-output-buffer-limit': 'normal 0 0 0 slave 268435456 67108864 60 pubsub 33554432 8388608 60',
 'replica-announce-port': '0',
 'tls-key-file': '',
 'no-appendfsync-on-rewrite': 'no',
 'oom-score-adj': 'no',
 'slaveof': '',
 'cluster-announce-human-nodename': '',
 'lfu-decay-time': '1',
 'tls-ciphersuites': '',
 'lazyfree-lazy-user-flush': 'no',
 'enable-debug-command': 'no',
 'save': '3600 1 300 100 60 10000',
 'hash-max-listpack-entries': '512',
 'server_cpulist': '',
 'pidfile': '/run/redis/redis-server.pid',
 'slave-ignore-maxmemory': 'yes',
 'cluster-preferred-endpoint-type': 'ip',
 'replica-lazy-flush': 'no',
 'tls-client-cert-file': '',
 'shutdown-timeout': '10',
 'replica-announce-ip': '',
 'set-proc-title': 'yes',
 'hash-max-ziplist-entries': '512',
 'list-compress-depth': '0',
 'rdb-save-incremental-fsync': 'yes',
 'min-slaves-max-lag': '10',
 'cluster-s

## Stockage clé valeurs

Pour stocker une valeur correspondant à une clé 

In [21]:
KEY = "name"
VALUE = "yourname"
redis_client.set(KEY, VALUE)

True

True montre que le stockage s'est bien déroulé. 

Pour récupérer la valeur stockée il suffit d'appeler la méthode `GET` sur cette clé

In [22]:
redis_client.get(KEY)

b'yourname'

Vous pouvez voir qu'à l'aide du `b` que la réponse n'est pas exactement ce que l'on a stocké, mais une représentation en bytes de notre chaine de charactères. 

In [23]:
assert redis_client.get(KEY) == VALUE, "Les valeurs ne correspondent pas"

AssertionError: Les valeurs ne correspondent pas

Pour cela, en python, il faut appeler la méthode `decode()` pour transformer des bytes en string 

In [12]:
assert redis_client.get(KEY).decode() == VALUE, "les valeurs ne correspondent pas"

On peut préciser au client de décoder les réponses et de les transformer directement en strings

In [13]:
redis_client = redis.StrictRedis(host='localhost', charset='utf-8', decode_responses=True)

In [14]:
assert redis_client.get(KEY) == VALUE

Les valeurs peuvent aussi être des entiers

In [15]:
KEY = "age"
VALUE = 20

redis_client.set(KEY, VALUE)

True

In [16]:
redis_client.get(KEY)

'20'

Vous pouvez alors utiliser la méthode `incr` pour incrémenter cet entier. Il faut faire attention puisque redis renvoie toujours les entiers comme des chaines de charactères.

In [17]:
redis_client.incr(KEY)

21

## Exemple 

In [18]:
from ipywidgets import widgets, interact
from IPython.display import display

In [19]:
KEY = "click_number"
DEFAULT_VALUE = 0

redis_client.set(KEY, DEFAULT_VALUE)

button = widgets.Button(description="Click me!")
def on_button_clicked(button):
    redis_client.incr(KEY)
    print(f"Le bouton à été cliqué {redis_client.get(KEY)} fois")
    button.description = "Click me more !"
    
button.on_click(on_button_clicked)
display(button)

Button(description='Click me!', style=ButtonStyle())

In [22]:
print(f"Tu as cliqué sur le bouton {redis_client.get(KEY)} fois !")

Tu as cliqué sur le bouton 10 fois !


## Les Listes

Redis permet aussi de stocker des valeurs de type `list`, pour cela nous avons à notre disposition un ensemble de méthodes :  

- La méthode `LPUSH`  permet d'ajouter un élément en tête de list la complexité est de O(1).  
- La méthode `RPUSH` permet d'ajouter un élément en queue de list la complexité est de O(1).  
- La méthode `LINSERT` permet d'ajouter un élément avant ou après la première occurence d'une certaine valeur, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `LSET` permet de modifier la valeur correspondant à l'index spécifié, la complexité est de O(N) à part pour le premier et le dernier élèment.
- La méthode `RPOP` permet de supprimer un élément en queue de list la complexité est de O(1).
- La méthode `LPOP` permet de supprimer un élément en tête de list la complexité est de O(1).
- La méthode `LINDEX` permet d'afficher un élément en fonction de son index.
- La méthode `LREM` permet de supprimer un élément en fonction de son index.
- La méthode `LLEN` permet de récupérer la taille d'une liste.

Nous allons toutes les tester.

In [23]:
redis_client.lpush('language_list', "kotlin")
redis_client.lpush('language_list', "python")

2

On peut aussi ajouter plusieurs éléments en même temps dans une liste

In [24]:
redis_client.lpush('language_list', "java", "c++")

4

In [25]:
f"La taille de la liste est {redis_client.llen('language_list')}"

'La taille de la liste est 4'

In [28]:
INDEX = 2
f"L'élément à l'index {INDEX} de la liste est '{redis_client.lindex('language_list', INDEX)}'"

"L'élément à l'index 2 de la liste est 'python'"

Le client renvoie la taille de la liste venant d'être mise à jour. On peut afficher tous les éléments de la liste en les retirant un par un.

In [29]:
while(redis_client.llen('language_list')!=0):

    print(redis_client.lpop('language_list'))

c++
java
python
kotlin


## Les dictionnaires 

Redis permet de stocker assez facilement des dictionnaires, on peut comme cela stocker des structures de données plus complexes. Pour cela, nous avons plusieurs méthodes : 

- La méthode `HMSET` permet d'ajouter un dictionnaire à une clé
- La méthode `HSET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGET` permet d'ajouter ou de modifier la valeur d'une clé d'un dictionnaire
- La méthode `HGETALL` permet de récupérer toutes les clés valeurs d'un dictionnaire.
- La méthode `HKEYS` permet de récupérer toutes les clés d'un dictionnaire
- La méthode `HVALS` permet de récupérer toutes les valeurs d'un dictionnaire

In [42]:
doc = {
    "name":"yourname",
    "age":20,
    "city":"paris"
}

In [43]:
redis_client.hset("user1", mapping=doc)

0

In [44]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'paris'}

On peut modifier uniquement une clé du dictionnaire

In [45]:
redis_client.hset("user1", "city", "lyon")

0

In [46]:
redis_client.hgetall("user1")

{'name': 'yourname', 'age': '20', 'city': 'lyon'}

## Les pipelines

In [47]:
import timeit

Lorsque l'on appelle une méthode depuis le client python, chaqu'une d'entre elles fait un appel à Redis, ce qui peut prendre du temps. Les pipelines permettent de réduire ce temps en stockant toutes les opérations et en les applicant une seule fois sur la base. Cela permet d'augmenter grandement les performances en réduisant les appels et le passage par le réseau. 

In [48]:
N = 10000

In [49]:
def store_key_values(N):
    for i in range(N):
        redis_client.set(f"key_{i}", f"value_{i}")

In [50]:
%%timeit
store_key_values(N=N)

1.19 s ± 29.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [51]:
def store_key_values_with_pipelines(N):
    p = redis_client.pipeline()
    for i in range(N):
        p.set(f"key_{i}", f"value_{i}")

In [52]:
%%timeit
store_key_values_with_pipelines(N=N)

4.22 ms ± 28.9 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


On voit alors que l'on peut diviser le temps par 100. Dans vos différents métiers les performances sont souvent très importantes

# Exercice

Utiliser redis pour gérer les commandes d'un restaurant de burgers, pour simplifier les choses les clients pourront choisir leurs ingrédients dans 3 types de produits :

- Pain: blanc, complet, aux céréales
- Garniture : salade, tomates , oignons, concombres
- Protéines : poulet, boeuf, agneau, tofu

Chaque ingrédient équivaut à un prix et un poids, avec chaque commande, vous devrez calculer le poids et le prix que les clients devront régler. 

1) Stocker les informations concernant les ingrédients dans redis   
2) Créer la méthode permettant de créer aléatoirement un burger  
3) Créer la méthode qui permet d'empiler dans redis les prochaines commandes (Réfléchir au type de données utilisé)  
4) Créer la méthode permettant de calculer le poids et le prix du burger  
5) Créer une boucle pour dépiler, réaliser et servir les burgers.  

Pour vous aider voici quelques données 

In [24]:
import random
# Connexion à Redis
r = redis.StrictRedis(host='localhost', port=6379, db=0)

In [25]:
pains = {
    "blanc":{
        "poid":100,
        "prix": 2
    },
    "complet":{
        "poid":90,
        "prix": 3
    },
    "cereales":{
        "poid":120,
        "prix": 4
    }
}

In [26]:
garniture = {
    "salade":{
        "poid":50,
        "prix": 1
    }, "tomates":{
        "poid":60,
        "prix": 2
    } , "oignons":{
        "poid":10,
        "prix": 1.5
    }, "concombres":{
        "poid":30,
        "prix": 3
    }
}

In [27]:
proteine = {
    "poulet":{
        "poid":100,
        "prix": 6
    }, "boeuf":{
        "poid":150,
        "prix": 5
    } , "agneau":{
        "poid":120,
        "prix": 7
    }, "tofu":{
        "poid":80,
        "prix": 8
    }
}

In [28]:
def make_burger():
    pain = random.choice(list(pains.keys()))
    garniture = random.choice(list(garniture.keys()))
    proteine = random.choice(list(proteine.keys()))
    
    burger = {
        'pain': pain,
        'garniture': garniture,
        'proteine': proteine
    }
    
    return burger


def make_order():
    burger = make_burger()
    r.rpush('orders', str(burger))
    return burger

def get_next_order():
    order = r.lpop('orders')  
    if order:
        return eval(order)  
    return None

def process_price_weight(burger):
    total_price = 0
    total_weight = 0
    
    for key, value in burger.items():
        ingredient_data = None
        if key == 'pain':
            ingredient_data = pains[value]
        elif key == 'garniture':
            ingredient_data = garniture[value]
        elif key == 'proteine':
            ingredient_data = proteine[value]
        
        total_price += ingredient_data['prix']
        total_weight += ingredient_data['poid']
    
    return total_price, total_weight


def process_burger():
    burger = get_next_order()
    if burger:
        price, weight = process_price_weight(burger)
        return burger, price, weight
    return None, None, None

In [30]:
while True:
    burger, price, weight = process_burger()
    if burger:
        print(f'Le burger {burger} de {weight} gr a été préparé et servi. Il faut régler {price}€')
    else:
        print("Plus de commandes à traiter.")
        break

Plus de commandes à traiter.
